# Problem Description

Consider the following equation:

$$\frac{\partial f}{\partial x}[x_i] \approx \frac{P_{r,i}(x_{i+\frac{1}{2}}) - P_{r,i-1}(x_{i-\frac{1}{2}})}{h} + O(h^r)$$

Where $r$ denotes the order of the interpolation approximation $P$, and $h$ denotes the step size.  To check the validity of this equation, let's take a consider an arbitrary, smooth function $f$ over a discrete domain $x_i$ where $ i \in [0,n-1]$ and n is the number of discrete points.  Hence, the step size is $$h=\frac{x_n-x_i}{n-1}$$

However, rather than reconstructing a polynomial interpolation of this function, let's just _directly_ calculate its value at all half points.  That is, let's just evaluate $$f(x_{i+\frac{1}{2}}) = f(x_i+\tfrac{h}{2}) $$

And use the following approximation:


$$\frac{\partial f}{\partial x}[x_i] \approx \frac{f(x_{i+\frac{1}{2}}) - f(x_{i-\frac{1}{2}})}{h}$$

In [128]:
import numpy as np
import pandas as pd

'''Set the input variables'''
x_min=0
x_max=1
n=5120
def f(x):
    return np.sin(2*np.pi*x)
def f_prime(x):
    return 2*np.pi*np.cos(2*np.pi*x)

'''Calculate additional parameters'''
x_vals,h=np.linspace(x_min,x_max,n,retstep=True)

# We pad the x values vector with an additional value to the beginning and end
x_vals=np.insert(x_vals,0,x_min-h)
x_vals=np.append(x_vals,x_max+h)

x_half_vals=x_vals+1/2*h

'''Create a Dataframe to store everything'''
DF=pd.DataFrame({'x':x_vals,'x_half':x_half_vals})

'''Evaluate the function at each i'''
DF['f']=f(DF.x)

'''Evaluate the function at each half point'''
DF['f_half']=f(DF.x_half)

'''Create a second half values column that is shifted backwards by i-1'''
DF['f_half_shift']=DF.f_half.shift()

Next let's take a look at our vectors to make sure things appear to be in order

In [129]:
print('h = ' +str(h))
print('------------------------------------------')
print('------------------------------------------\n')
print('FIRST FIVE ROWS')
print(DF.head(5))
print('\nLAST FIVE ROWS')
print(DF.tail(5))


h = 0.0001953506544246923
------------------------------------------
------------------------------------------

FIRST FIVE ROWS
          x    x_half         f    f_half  f_half_shift
0 -0.000195 -0.000098 -0.001227 -0.000614           NaN
1  0.000000  0.000098  0.000000  0.000614     -0.000614
2  0.000195  0.000293  0.001227  0.001841      0.000614
3  0.000391  0.000488  0.002455  0.003069      0.001841
4  0.000586  0.000684  0.003682  0.004296      0.003069

LAST FIVE ROWS
             x    x_half             f    f_half  f_half_shift
5117  0.999414  0.999512 -3.682265e-03 -0.003069     -0.004296
5118  0.999609  0.999707 -2.454846e-03 -0.001841     -0.003069
5119  0.999805  0.999902 -1.227424e-03 -0.000614     -0.001841
5120  1.000000  1.000098 -2.449294e-16  0.000614     -0.000614
5121  1.000195  1.000293  1.227424e-03  0.001841      0.000614


Next, we employ our derivative approximation technique by subtracting *f\_half\_shift* from *f_half* and dividing by $h$. Also, let's calcuate the actual derivattive analytically and create a column for this value.

In [130]:
DF['f_prime_approx'] = (DF.f_half-DF.f_half_shift)/h
DF['f_prime_exact'] = f_prime(DF.x)

'''Let's calculate the absolute error between these two values'''
DF['Error'] = abs(DF.f_prime_approx - DF.f_prime_exact)

'''Let's print out the first five rows'''
DF.head(5)

,x,x_half,f,f_half,f_half_shift,f_prime_approx,f_prime_exact,Error
0,-0.000195,-0.000098,-0.001227,-0.000614,NaN,NaN,6.283181,NaN
1,0.000000,0.000098,0.000000,0.000614,-0.000614,6.283185,6.283185,3.944192e-07
2,0.000195,0.000293,0.001227,0.001841,0.000614,6.283180,6.283181,3.944189e-07
3,0.000391,0.000488,0.002455,0.003069,0.001841,6.283166,6.283166,3.944181e-07
4,0.000586,0.000684,0.003682,0.004296,0.003069,6.283142,6.283143,3.944166e-07


Since the first and last points are not part of our domain, let's exclude them from our results

In [131]:
DF=DF[1:-1]

### First Five Rows

In [132]:
DF.head(5)

,x,x_half,f,f_half,f_half_shift,f_prime_approx,f_prime_exact,Error
1,0.000000,0.000098,0.000000,0.000614,-0.000614,6.283185,6.283185,3.944192e-07
2,0.000195,0.000293,0.001227,0.001841,0.000614,6.283180,6.283181,3.944189e-07
3,0.000391,0.000488,0.002455,0.003069,0.001841,6.283166,6.283166,3.944181e-07
4,0.000586,0.000684,0.003682,0.004296,0.003069,6.283142,6.283143,3.944166e-07
5,0.000781,0.000879,0.004910,0.005523,0.004296,6.283109,6.283110,3.944145e-07


---
### Last Five Rows

In [133]:
DF.tail(5)

,x,x_half,f,f_half,f_half_shift,f_prime_approx,f_prime_exact,Error
5116,0.999219,0.999316,-4.909678e-03,-0.004296,-0.005523,6.283109,6.283110,3.944173e-07
5117,0.999414,0.999512,-3.682265e-03,-0.003069,-0.004296,6.283142,6.283143,3.944103e-07
5118,0.999609,0.999707,-2.454846e-03,-0.001841,-0.003069,6.283166,6.283166,3.944209e-07
5119,0.999805,0.999902,-1.227424e-03,-0.000614,-0.001841,6.283180,6.283181,3.944218e-07
5120,1.000000,1.000098,-2.449294e-16,0.000614,-0.000614,6.283185,6.283185,3.944130e-07


## Part Two - Error Analysis

There is little utility in this analysis unless we repeate the process incrementally increasing the number of points in the domain.  To accomplish this, we need to rewrite the above code in such a way that we can loop through many different $n$ values and record the maximum error for each one. 

In [134]:
import numpy as np
import pandas as pd

def approximate(f,f_prime,x_min,x_max,n):
    '''Calculate additional parameters'''
    x_vals,h=np.linspace(x_min,x_max,n,retstep=True)

    # We pad the x values vector with an additional value to the beginning and end
    x_vals=np.insert(x_vals,0,x_min-h)
    x_vals=np.append(x_vals,x_max+h)

    x_half_vals=x_vals+1/2*h

    '''Create a Dataframe to store everything'''
    DF=pd.DataFrame({'x':x_vals,'x_half':x_half_vals})

    '''Evaluate the function at each i'''
    DF['f']=f(DF.x)

    '''Evaluate the function at each half point'''
    DF['f_half']=f(DF.x_half)

    '''Create a second half values column that is shifted backwards by i-1'''
    DF['f_half_shift']=DF.f_half.shift()

    '''Evaluate the Derivatives'''
    DF['f_prime_approx'] = (DF.f_half-DF.f_half_shift)/h
    DF['f_prime_exact'] = f_prime(DF.x)

    '''Let's calculate the absolute error between these two values'''
    DF['Error'] = abs(DF.f_prime_approx - DF.f_prime_exact)
    DF=DF[1:-1]
    
    return DF.Error.max()

In [137]:
x_start=0
x_stop=1
n_points=[10,20,40,80,160,320,640,1280,5120,10240]
Error=np.zeros(len(n_points))

def func(x):
    return np.sin(2*np.pi*x)
def func_prime(x):
    return 2*np.pi*np.cos(2*np.pi*x)

i=0
for points in n_points:
    Error[i]=approximate(f=func, f_prime=func_prime, x_min=x_start, x_max=x_stop,n=points)
    i=i+1

ErrorDF=pd.DataFrame({'n':n_points,'ErrorMax':Error})
ErrorDF['Ratio'] = ErrorDF.ErrorMax/ErrorDF.ErrorMax.shift(-1)

ErrorDF

,n,ErrorMax,Ratio
0,10,1.268227e-01,4.435776
1,20,2.859088e-02,4.208906
2,40,6.792947e-03,4.102215
3,80,1.655922e-03,4.050552
4,160,4.088139e-04,4.025138
5,320,1.015652e-04,4.012534
6,640,2.531198e-05,4.006259
7,1280,6.318109e-06,16.018663
8,5120,3.944218e-07,4.000404
9,10240,9.859547e-08,NaN


## Discussion

What are the implications of this analysis?  We are using the _exact_ value of the function at each half point, so this eliminates any error associated with the polynomial reconstruction. At this limit, our error term decreases by a factor of 4 as we double the number of points.  This is exactly the behaviour I have been observing with my code.   No matter what order of $P$ approximation I use, I still observe the Error decreasing by _4_  not $O(h^r)$ as indicated by the equation. 